# Image Style Transferの実装

--------------------------------
Gatys, Ecker, and Bethge(2016)の画像スタイル変換を実装する。

【参考文献】  
L. A. Gatys, A. S. Ecker, and M. Bethge,  
Image style transfer using convolutional neural networks,  
In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, pages, 2414-2423, 2016.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
CONTENT_FILE = '/home/ishiyama/image_style_transfer/image/input/test_input_01.JPG'
STYLE_FILE = '/home/ishiyama/image_style_transfer/image/style/test_style_01.jpg'

In [3]:
def read_images_as_jpeg(file):
    
    """
    JPEG Image Reader
    
    This function reads the content and style images as JPEG format.
    These image data must be square for now, different height and
    width will be able to supplied for future.
    
    Args:
        file : str. A path of the image file.
    
    Returns:
        A tuple. Each Elements are Tensor object of the read images.
    """
    
    filename_queue = tf.train.string_input_producer([file])
    reader = tf.WholeFileReader()
    key, value = reader.read(filename_queue)
    image = tf.image.decode_jpeg(value)

    # Convert read image data to RGB representation.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    tf.train.start_queue_runners(sess)
    image_rgb = sess.run(image)
    height, width, channels = image_rgb.shape

    return image_rgb.reshape([1, height, width, channels])

In [4]:
image = read_images_as_jpeg(file=CONTENT_FILE)

# VGGを実装する
------------------------
画像の特徴量を抽出するアルゴリズムにはSimonyan and Zisserman(2015)で提案されたCNN(VGG19)の畳込み層とプーリング層が使われている。  
ここでは、「TensorFlowで学ぶディープラーニング入門」の多層CNNの実装を参考にVGG19を構築する。  

【参考文献】  
K. Simonyan and A. Zisserman, Very Deep Convolutional Networks For Large-Scale Image Recognition, arXiv: 1409.1556v6, 2015  
中井悦司, TensorFlowで学ぶディープラーニング入門〜畳み込みニューラルネットワーク徹底解説, マイナビ出版, 2016

### 畳み込み層を実装する

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


def calculate_convolutional_layer(x, filter_height, filter_width, output_channels):

    """
    Executeing a convolutional layer task.
    
    Args:
        x                     : An image data.
        filter_height   (int) : A height of each filters.
        filter_width    (int) : A width of each filters.
        output_channels (int) : A number of channels which must be outputed.

    Returns:
        An activation of an convolutional layer.
    """

    if ((not isinstance(filter_height, int))
        or (not isinstance(filter_width, int))
        or (not isinstance(output_channels, int))):
        raise TypeError('"filter_height" and "filter_width" and "output_channels" '
                        'must be integer.')

    input_channels = int(x.shape[-1])
    W = tf.Variable(
        tf.truncated_normal(
            shape=[filter_height,
                   filter_width,
                   input_channels,
                   output_channels],
            stddev=0.1
        )
    )
    h = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    b = tf.Variable(tf.constant(0.1, shape=[output_channels]))
    convoluted_image = tf.nn.relu(h + b)

    return convoluted_image

In [6]:
x = tf.placeholder(tf.float32, [1, 1200, 1600, 3])
test = calculate_convolutional_layer(
    x=x,
    filter_height=3,
    filter_width=3,
    output_channels=1
)
sess = tf.InteractiveSession()
# tf.Session()だと、sess.runで返ってくる行列の要素がすべて0だった。
# TODO: Sessionメソッド と InteractiveSessionメソッドの違いを調べる
# sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(test, feed_dict={x: image})

array([[[[  0.        ],
         [  0.        ],
         [  0.        ],
         ..., 
         [  0.        ],
         [  0.        ],
         [ 14.7912941 ]],

        [[  0.        ],
         [ 19.95758629],
         [ 18.51756859],
         ..., 
         [ 19.386549  ],
         [ 19.57115746],
         [ 65.23432159]],

        [[  0.        ],
         [ 18.29010201],
         [ 17.54481506],
         ..., 
         [ 19.24207115],
         [ 19.96671677],
         [ 65.07197571]],

        ..., 
        [[  0.        ],
         [ 21.11691093],
         [ 21.05081367],
         ..., 
         [ 24.52385712],
         [ 24.15638542],
         [ 88.45456696]],

        [[  0.        ],
         [ 21.07484627],
         [ 22.14480591],
         ..., 
         [ 24.70388031],
         [ 24.40051079],
         [ 88.44583893]],

        [[  8.90180397],
         [  6.8230834 ],
         [  5.30810308],
         ..., 
         [  9.68836594],
         [  9.75821686],
         [ 

### Maxプーリング層を実装する

In [7]:
def calculate_max_pooling_layer(x, ksize, strides):

    """Wrapper function of tf.nn.max_pool.
    
    Args:
        x       : A Tensor produced by calculate_convolutional_layer.
        ksize   : A list of ints that has length >= 4. The size of
                  the window for each dimension of the input tensor.
        strides : A list of ints that has length >= 4. The stride
                  of the sliding window for each dimension of the
                  input tensor.
    
    Returns:
        A pooled image.
    """

    pooled_image = tf.nn.max_pool(x, ksize=ksize, strides=strides, padding='SAME')

    return pooled_image

### 畳込みとプーリング処理の途中経過を保持するクラスを実装する

In [8]:
FILTER_CONF = {
    'height': 3,
    'width': 3,
    'channels': 1,
    'num': 1
}

class ConvNetProgressHolder(object):

    """Holder of convoluted images and pooled image.
    
    This class is used like the struct of C language.
    This has no methods.
    
    Attributes:
        input_data (Tensor) : An image that is applied to convolution and pooling.
        conv (list)         : The list of convoluted images, each images are Tensor objects.
        pool (Tensor)       : A image that is pooled after convolutional layer.
    """

    def __init__(self):

        self.input_data = None
        self.conv = []
        self.pool = None

    
def apply_vgg_network_unit(x, channels, num_conv):

    """Apply VGG Network From a Convolutional Layer to Max Pooling Layer.

    Table 1 of Simonyan and Zisserman(2015) is separated by 5 parts,
    each parts is from an input data or a pooled data at previous part
    to a maxpool.
    This function provides to apply a that part.
    This will apply recursively.
    
    Args:
        x (Tensor)     : An input data or A Max pooled data returned by this function.
        channels (int) : A number of channels described at Table 1 of
                         Simonyan and Zisserman(2015).
        num_conv (int) : A number of applying covolutional layers.
                         See Simonyan and Zisserman(2015) for detail.

    Returns:
        A ConvNetProgressHolder object.
    """

    if num_conv < 2:
        raise ValueError('num_conv must be >= 2.')

    print(x)

    conv_holder = ConvNetProgressHolder()
    conv_holder.input_data = x

    conv = calculate_convolutional_layer(
        x=conv_holder.input_data,
        filter_height=FILTER_CONF['height'],
        filter_width=FILTER_CONF['width'],
        output_channels=channels
    )
    conv_holder.conv.append(conv)

    for i in range(1, num_conv):
        conv = calculate_convolutional_layer(
            x=conv_holder.conv[i - 1],
            filter_height=FILTER_CONF['height'],
            filter_width=FILTER_CONF['width'],
            output_channels=channels
        )
        conv_holder.conv.append(conv)

    conv_holder.pool = calculate_max_pooling_layer(
        x=conv_holder.conv[i - 1],
        ksize=[1, 1, 1, 1],
        strides=[1, 1, 1, 1]
    )

    return conv_holder


### VGGの畳込みとプーリング層を構築する

# 処理が重たいので実行に注意する。
※　このマシンでは1段目が限界

In [10]:
x = tf.placeholder(tf.float32, [1, 1200, 1600, 3])
print('関数の外: ', x)
unit1 = apply_vgg_network_unit(x=x, channels=64, num_conv=2)
# print('関数の外: ', unit1.pool)
# unit2 = apply_vgg_network_unit(x=unit1.pool, channels=128, num_conv=2)
# print('関数の外: ', unit2.pool)
# unit3 = apply_vgg_network_unit(x=unit2.pool, channels=256, num_conv=4)
# print('関数の外: ', unit3.pool)
# unit4 = apply_vgg_network_unit(x=unit3.pool, channels=512, num_conv=4)
# print('関数の外: ', unit4.pool)
# unit5 = apply_vgg_network_unit(x=unit4.pool, channels=512, num_conv=4)
sess = tf.InteractiveSession()
# sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(unit1.pool, feed_dict={x: image})

関数の外:  Tensor("Placeholder_2:0", shape=(1, 1200, 1600, 3), dtype=float32)
Tensor("Placeholder_2:0", shape=(1, 1200, 1600, 3), dtype=float32)


array([[[[   0.        ,    5.4706769 ,    0.        , ...,    0.        ,
            74.99827576,  105.98683929],
         [   0.        ,    0.        ,    0.        , ...,    0.        ,
            73.5799942 ,   93.82923126],
         [   0.        ,    0.        ,    0.        , ...,    0.        ,
            73.72968292,   92.15812683],
         ..., 
         [   0.        ,    0.        ,    0.        , ...,    0.        ,
            54.11470032,   69.58145905],
         [   0.        ,    0.        ,    0.        , ...,    0.        ,
            54.61402893,   69.3035965 ],
         [   0.        ,    0.        ,    3.19779348, ...,    0.        ,
            41.23967743,    0.        ]],

        [[   0.        ,   51.46072388,    0.        , ...,    0.        ,
           108.52433777,   32.36486816],
         [   0.        ,   24.044487  ,    0.        , ...,    4.5194869 ,
            96.15283966,    9.20607948],
         [   0.        ,   23.03251648,    0.        , 